In [67]:
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
stopwords = set(english_stopwords)
import json
import pandas as pd
from tqdm import tqdm

In [68]:
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler

# read card_deck from ppn_deck.json file
with open("ppn_deck.json", "r") as read_file:
    card_deck = json.load(read_file)

def revise_point_values(card_deck, max_points=100):
    # Initialize a dictionary to store the word counts for each card
    word_counts = {}
    # Initialize a list to store the word counts for each card
    card_word_counts = []
    
    # Get a list of English stopwords
    stop_words = stopwords.words("english")
    
    # Iterate through each card in the card deck
    card_id = 0
    for card in tqdm(card_deck):
        # Get the card's id from the iteration
        # Get the summary for the card
        summary = card['summary']
        # Split the summary into words. Currently it is a tuple, get the second element.
        summary_text = summary[1]
        # Split the summary into words
        words = summary_text.split()
        # Remove the stop words from the list of words
        nonstop_words = [word for word in words if word not in stop_words]
        # Calculate the number of non-stop words
        word_count = len(nonstop_words)
        # Add the word count to the dictionary
        word_counts[card_id] = word_count
        # Append the word count to the list of card word counts
        card_word_counts.append(word_count)
        # Increment the card id
        card_id += 1
    # Use MinMaxScaler to scale the word counts to a range of 1 to 100
    scaler = MinMaxScaler(feature_range=(1, int(max_points)))
    scaled_word_counts = scaler.fit_transform(pd.DataFrame(card_word_counts))
    # Iterate through each card in the card deck
    card_id = 0
    for card in tqdm(card_deck):
        #print(card)
        # Get the card's id from the iteration
        # Get the scaled word count for the card
        scaled_word_count = scaled_word_counts[card_id]
        # Get the card's point value if it exists
        point_value = card['point_value'] if 'point_value' in card else 0
        rounded_value = round(scaled_word_count[0])

        # Update the card's point value to the scaled word count
        card['point_value'] = rounded_value
        # Increment the card id
        card_id += 1
    return card_deck
    


To fix this, you can reshape card_word_counts to a 2D array using the reshape method of the numpy library, like this: card_word_counts.reshape(-1, 1). This will transform card_word_counts into a 2D array with a single column and as many rows as there are elements in card_word_counts.

You can then pass the reshaped array to the fit method of the MinMaxScaler object.

You will also need to make the same change to the line card_word_counts_scaled = scaler.fit_transform(card_word_counts.reshape(-1, 1)) in order to avoid the same error when you try to fit the scaler to the data.

Finally, in the for loop where you update the point value for each card, you should use card_word_counts_scaled[card_id] instead of card_word_counts_scaled[word_count]. This is because card_word_counts_scaled is a list of scaled word counts, and you need to use the current card_id to index into the list

In [69]:
# revise point values
card_deck = revise_point_values(card_deck,20)


100%|██████████| 3795/3795 [00:00<00:00, 702382.12it/s]
